# Lesson 15: EM + Gaussian Mixture Models## Objectives- Implement EM for a 2D Gaussian mixture.- Track log-likelihood over iterations.- Visualize component responsibilities.

## From the notesEM alternates:- E-step: compute responsibilities \(\gamma_{ik}\).- M-step: update mixture weights, means, covariances.

## IntuitionEM treats component assignments as latent variables and iteratively improves the likelihood.

## DataWe generate data from two Gaussian components.

In [ ]:
import numpy as npimport matplotlib.pyplot as pltnp.random.seed(42)

In [ ]:
# Synthetic GMM datam = 200X1 = np.random.multivariate_normal([-2,0], np.eye(2)*0.6, size=m//2)X2 = np.random.multivariate_normal([2,1], np.eye(2)*0.6, size=m//2)X = np.vstack([X1, X2])

## Implementation: EM for GMM

In [ ]:
def gaussian_pdf(X, mean, cov):    d = X.shape[1]    cov_inv = np.linalg.pinv(cov)    diff = X - mean    exp_term = np.exp(-0.5 * np.sum(diff @ cov_inv * diff, axis=1))    denom = np.sqrt((2*np.pi)**d * (np.linalg.det(cov) + 1e-9))    return exp_term / denomdef em_gmm(X, K=2, num_iters=20):    m, d = X.shape    # init    means = X[np.random.choice(m, K, replace=False)]    covs = [np.eye(d) for _ in range(K)]    pis = np.ones(K) / K    loglik = []    for _ in range(num_iters):        # E-step        resp = np.zeros((m, K))        for k in range(K):            resp[:, k] = pis[k] * gaussian_pdf(X, means[k], covs[k])        resp = resp / resp.sum(axis=1, keepdims=True)        # M-step        Nk = resp.sum(axis=0)        for k in range(K):            means[k] = (resp[:, k][:, None] * X).sum(axis=0) / Nk[k]            diff = X - means[k]            covs[k] = (resp[:, k][:, None, None] * (diff[:, :, None] * diff[:, None, :])).sum(axis=0) / Nk[k]            covs[k] += 1e-6 * np.eye(d)        pis = Nk / m        ll = np.sum(np.log(resp.sum(axis=1) + 1e-9))        loglik.append(ll)    return means, covs, pis, resp, np.array(loglik)

## Experiments

In [ ]:
means, covs, pis, resp, loglik = em_gmm(X, K=2)

## Visualizations

In [ ]:
plt.figure(figsize=(6,4))plt.scatter(X[:,0], X[:,1], c=resp[:,0], cmap="coolwarm", alpha=0.7)plt.scatter([m[0] for m in means], [m[1] for m in means], color="black", marker="x", s=100)plt.xlabel("x1")plt.ylabel("x2")plt.title("GMM responsibilities")plt.show()plt.figure(figsize=(6,4))plt.plot(loglik)plt.xlabel("iteration")plt.ylabel("log-likelihood")plt.title("EM convergence")plt.show()

## Takeaways- EM alternates soft assignments and parameter updates.- Log-likelihood should increase monotonically.

## Explain it in an interview- Describe E-step/M-step updates and latent variables.- Mention sensitivity to initialization.

## Exercises1. Increase to K=3 components and compare results.2. Implement a diagonal-covariance version.3. Plot mixture contours to visualize Gaussian shapes.